- [WSGI Server实现](#WSGI-Server实现)
  * [WSGIServer class](#WSGIServer-class)
  * [WSGIRequestHandler class](#WSGIRequestHandler-class)
  * [ServerHandler class](#ServerHandler-class)
    + [BaseHandler class](#BaseHandler-class)
    + [SimpleHandler class](#SimpleHandler-class)
    + [ServerHandler class](#serverhandler)
  * [WSGI application](#WSGI-application)
  * [Summary](#Summary)
  * [Reference](#Reference)
  
# WSGI Server实现

与HTTP server一样，要实现完整的http server，必须有Server class与Handler class。

## WSGIServer class
```Python
class WSGIServer(HTTPServer):
    """BaseHTTPServer that implements the Python WSGI protocol"""
    application = None

    def server_bind(self):
        """Override server_bind to store the server name."""
        HTTPServer.server_bind(self)
        self.setup_environ()

    def setup_environ(self):
        # Set up base environment
        env = self.base_environ = {}
        env['SERVER_NAME'] = self.server_name
        env['GATEWAY_INTERFACE'] = 'CGI/1.1'
        env['SERVER_PORT'] = str(self.server_port)
        env['REMOTE_HOST']=''
        env['CONTENT_LENGTH']=''
        env['SCRIPT_NAME'] = ''

    def get_app(self):
        return self.application

    def set_app(self,application):
        self.application = application
```
WSGIServer继承HTTPServer，与HTTPServer唯一的区别就是，它需要保存WSGi application实例并且设置env变量。

## WSGIRequestHandler class

WSGIRequestHandler的继承BaseHTTPRequestHandler，并override了handle()函数。
```Python
class WSGIRequestHandler(BaseHTTPRequestHandler):
    def handle(self):
        """Handle a single HTTP request"""
        # Whether the request length is valid.
        self.raw_requestline = self.rfile.readline(65537)
        if len(self.raw_requestline) > 65536:
            self.requestline = ''
            self.request_version = ''
            self.command = ''
            self.send_error(414)
            return

        #　Parse the http request content.
        if not self.parse_request(): # An error code has been sent, just exit
            return

        # Let the really handler to handle with parsed request.
        handler = ServerHandler(
            self.rfile, self.wfile, self.get_stderr(), self.get_environ()
        )
        handler.request_handler = self      # backpointer for logging
        handler.run(self.server.get_app())
    
    # more methods...
```
可以看出，WSGIRequestHandler的处理流程与BaseHTTPRequestHandler类似，它判断request是否合法，解析http request内容，实例化一个ServerHandler类去
执行真正的处理逻辑。

## ServerHandler class
上面提到，ServerHandler是用于真正处理解析后的request的。它的继承关系是： ServerHandler <- SimpleHandler <- BaseHandler
从父类向下一个个看，首先是BaseHandler。
### BaseHandler class
BaseHandler的实现遵循WSGI server规范，即:
1. 提供了start_response函数.
```Python
class BaseHandler:
    def start_response(self, status, headers,exc_info=None):
        """'start_response()' callable as specified by PEP 3333"""
        if exc_info:
            try:
                if self.headers_sent:
                    # Re-raise original exception if headers sent
                    raise exc_info[0](exc_info[1]).with_traceback(exc_info[2])
            finally:
                exc_info = None        # avoid dangling circular ref
        elif self.headers is not None:
            raise AssertionError("Headers already set!")

        self.status = status
        self.headers = self.headers_class(headers)
        status = self._convert_string_type(status, "Status")
        assert len(status)>=4,"Status must be at least 4 characters"
        assert status[:3].isdigit(), "Status message must begin w/3-digit code"
        assert status[3]==" ", "Status message must have a space after code"

        if __debug__:
            for name, val in headers:
                name = self._convert_string_type(name, "Header name")
                val = self._convert_string_type(val, "Header value")
                assert not is_hop_by_hop(name),"Hop-by-hop headers not allowed"

        return self.write
    
    # more methods...
```
该函数实现比较简单，主要是设置http response的code和header，以及一些错误处理。
2. 调用callable application.
```Python
class BaseHandler:
    def run(self, application):
        """Invoke the application"""
        # Note to self: don't move the close()!  Asynchronous servers shouldn't
        # call close() from finish_response(), so if you close() anywhere but
        # the double-error branch here, you'll break asynchronous servers by
        # prematurely closing.  Async servers must return from 'run()' without
        # closing if there might still be output to iterate over.
        try:
            self.setup_environ()
            self.result = application(self.environ, self.start_response)
            self.finish_response()
        except:
            try:
                self.handle_error()
            except:
                # If we get an error handling an error, just give up already!
                self.close()
                raise   # ...and let the actual server figure it out.
    # more methods...
```
这里，
    * setup_environ函数设置wsgi协议中规定的环境变量，如input stream, output stream等。因此，application根据env可以获得完整的request。
    * application函数是让web application处理request，并拿到返回的body内容。
    * finish_response是将header和body发回给http client。

当然，在BaseHandler的finish_response()中并没有规定怎么把response body发给client，而是提供了一个未实现的的\_write()方法让子类去实现。

### SimpleHandler class
于是，SimpleHandler为了适配WSGIRequestHandler（因为WSGIRequestHandler是socketserver.StreamRequestHandler的子类，而socketserver.StreamRequestHandler里将socket读写用文件读写方式来操作），实现了一种类似文件读写的socket发送数据方式。
```Python
class SimpleHandler(BaseHandler):
    def _write(self,data):
        result = self.stdout.write(data)
        if result is None or result == len(data):
            return
        from warnings import warn
        warn("SimpleHandler.stdout.write() should not do partial writes",
            DeprecationWarning)
        while True:
            data = data[result:]
            if not data:
                break
            result = self.stdout.write(data)
    
    # more methods...
```
非常简单，就是向写文件一样把数据写下来。

### ServerHandler class<span id="serverhandler"></span>
ServerHandler类就更简单了，它overrride了close方法，为了记一下log。。
```Python
class ServerHandler(SimpleHandler):
    server_software = software_version

    def close(self):
        try:
            self.request_handler.log_request(
                self.status.split(' ',1)[0], self.bytes_sent
            )
        finally:
            SimpleHandler.close(self)
```

至此，一个简单的WSGI server就实现了，麻雀虽小，五脏俱全，所有该涉及到的内容都涉及到了。

## WSGI application
为了完整性，顺便介绍一下简单的WSGI application，如下函数：
```Python
def demo_app(environ,start_response):
    from io import StringIO
    stdout = StringIO()
    print("Hello world!", file=stdout)
    print(file=stdout)
    h = sorted(environ.items())
    for k,v in h:
        print(k,'=',repr(v), file=stdout)
    start_response("200 OK", [('Content-Type','text/plain; charset=utf-8')])
    return [stdout.getvalue().encode("utf-8")]
```
该application将"Hello world!"以及所有的WSGI环境变量作为body返回。

启动一下WSGI server试试:
```Python
def make_server(
    host, port, app, server_class=WSGIServer, handler_class=WSGIRequestHandler
):
    """Create a new WSGI server listening on `host` and `port` for `app`"""
    server = server_class((host, port), handler_class)
    server.set_app(app)
    return server

if __name__ == '__main__':
    with make_server('', 8000, demo_app) as httpd:
        sa = httpd.socket.getsockname()
        print("Serving HTTP on", sa[0], "port", sa[1], "...")
        import webbrowser
        webbrowser.open('http://localhost:8000/xyz?abc')
        httpd.handle_request()
```
运行后，浏览器应该会打印出完整的WSGI环境变量啦！

## Summary
在HTTP server实现中，request交给Handler处理，没有涉及到application的概念。而在WSGI server实现中，request交给application处理，因此会出现server与application通信的问题，这就是WSGI规范出现的原因。


## Reference
* [PEP 3333 -- Python Web Server Gateway Interface](https://www.python.org/dev/peps/pep-3333/)
